In [3]:
import pandas as pd
import shutil
import gzip

import os

In [2]:
def parse_fasta(protein_fasta):
    id = protein_fasta.split()[0]
    sequence = ''.join(protein_fasta.split("\n")[1:])
    return (id, sequence)

def get_pdb(accession_id):
    if not os.path.isfile(f"./../data/alphafold/pdb_HUMAN/AF-{accession_id}-F1-model_v1.pdb"):
        with gzip.open(f"./../data/alphafold/pdb_HUMAN/AF-{accession_id}-F1-model_v1.pdb.gz", "rb") as gz:
            with open(f"./../data/alphafold/pdb_HUMAN/AF-{accession_id}-F1-model_v1.pdb", "wb") as pdb:
                shutil.copyfileobj(gz, pdb)

    shutil.copyfile(
        f"./../data/alphafold/pdb_HUMAN/AF-{accession_id}-F1-model_v1.pdb",
        f"./../data/alphafold/pdb_HUMAN/AF-{accession_id}-F1-model_v1.pdb"
    )

def get_pdbs(fasta_file):
    with open(fasta_file, "r") as f:
        db_fasta = ("\n" + f.read()).split("\n>")[1:]
        for protein in db_fasta:
            protein_id, sequence = parse_fasta(protein)
            clean_id = protein_id.split("|")[1]
            get_pdb(clean_id)